In [1]:
%sh
sbt assembly


In [2]:
%sh
curl -n \
-F filedata=@"target/scala-2.10/ScalaSparkTemplate-assembly-0.2-SNAPSHOT.jar" \
-F path="/docs/tutorial.jar" \
-F overwrite=true \
https://YOURACCOUNT.cloud.databricks.com/api/2.0/dbfs/put

In [3]:
%sh
curl -n \
https://YOURACCOUNT.cloud.databricks.com/api/2.0/clusters/spark-versions

In [4]:
%sh
curl -n \
-X POST -H 'Content-Type: application/json' \
-d '{
      "name": "my example job",
      "new_cluster": {
        "spark_version": "2.0.1-db1-scala2.10",
        "node_type_id": "r3.xlarge",
        "aws_attributes": {"availability": "ON_DEMAND"},
        "num_workers": 1
        },
     "libraries": [{"jar": "dbfs:/docs/tutorial.jar"}],
     "spark_jar_task": {
        "main_class_name":"databricks.examples.ExampleClass",
        "parameters": "hello there"
        }
}' \
https://YOURACCOUNT.cloud.databricks.com/api/2.0/jobs/create

In [5]:
%sh
curl -n \
-X POST -H 'Content-Type: application/json' \
-d '{
      "job_id": 84005,
      "jar_params": ["hello", "params", "that are overwritten at run time"]
}' \
https://YOURACCOUNT.cloud.databricks.com/api/2.0/jobs/run-now

In [6]:
%sh
curl -n https://YOURACCOUNT.cloud.databricks.com/api/2.0/jobs/runs/get?run_id=786200

In [7]:
%sh
import json
import base64
import urllib

ACCOUNT = 'YOURACCOUNT'
USER = 'USERNAME'
PASSWORD = 'PASSWORD'
BASE_URL = 'https://%s:%s@%s.cloud.databricks.com/api/2.0/dbfs/' %\
    (urllib.quote(USER), urllib.quote(PASSWORD), ACCOUNT)

def dbfs_rpc(action, body):
    """ A helper function to make the DBFS API request, request/response is encoded/decoded as JSON """
    res = urllib.urlopen(BASE_URL + action, json.dumps(body)).read()
    return json.loads(res)

# Create a handle which will be used to add blocks
handle = dbfs_rpc("create", {"path": "/temp/upload_large_file", "overwrite": "true"})['handle']
with open('/a/local/file') as f:
    while True:
        # A block can be at most 1MB
        block = f.read(1 << 20)
        if not block:
            break
        data = base64.standard_b64encode(block)
        dbfs_rpc("add-block", {"handle": handle, "data": data})
# close the handle to finish uploading
dbfs_rpc("close", {"handle": handle})

In [8]:
%sql
ALTER (DATABASE|SCHEMA) db_name SET DBPROPERTIES (key1=val1, ...)

In [9]:
%sql
CREATE TABLE boxes (width INT, length INT, height INT) USING CSV

CREATE TEMPORARY TABLE boxes
    (width INT, length INT, height INT)
    USING PARQUET
    OPTIONS ('compression'='snappy')

CREATE TABLE rectangles
    USING PARQUET
    PARTITIONED BY (width)
    CLUSTERED BY (length) INTO 8 buckets
    AS SELECT * FROM boxes

CREATE OR REPLACE TEMPORARY VIEW temp_rectangles
    AS SELECT * FROM boxes